In [15]:
import requests
from urllib import quote

API_HOST = 'https://api.yelp.com'
BUSINESS_PATH = '/v3/businesses/'
BUSINESS_MATCH_PATH = '/v3/businesses/matches'

In [11]:
api_key = '[You API Key]'

In [20]:
def request(host, path, api_key, url_params=None):
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }

    print(u'Querying {0} ...'.format(url))

    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()

def get_business(api_key, business_id):
    """
    Get business info through business id
    """
    business_path = BUSINESS_PATH + business_id

    return request(API_HOST, business_path, api_key)

def match_business(api_key, params):
    """
    Get business info through business name
    """
    business_path = BUSINESS_MATCH_PATH

    return request(API_HOST, business_path, api_key, params)

In [22]:
params = {
    'name': 'bottega-louie-los-angeles',
    'address1': '700 S Grand Ave',
    'city': 'Los Angeles',
    'state': 'CA',
    'country': 'US'
}

match_business(api_key, params)

Querying https://api.yelp.com/v3/businesses/matches ...


{u'businesses': [{u'alias': u'bottega-louie-los-angeles',
   u'coordinates': {u'latitude': 34.0469300995766,
    u'longitude': -118.256601457672},
   u'display_phone': u'(213) 802-1470',
   u'id': u'TkFEKhsCixPWlShULKvMdQ',
   u'location': {u'address1': u'700 S Grand Ave',
    u'address2': None,
    u'address3': u'',
    u'city': u'Los Angeles',
    u'country': u'US',
    u'display_address': [u'700 S Grand Ave', u'Los Angeles, CA 90017'],
    u'state': u'CA',
    u'zip_code': u'90017'},
   u'name': u'Bottega Louie',
   u'phone': u'+12138021470'}]}

In [23]:
get_business(api_key, 'TkFEKhsCixPWlShULKvMdQ')

Querying https://api.yelp.com/v3/businesses/TkFEKhsCixPWlShULKvMdQ ...


{u'alias': u'bottega-louie-los-angeles',
 u'categories': [{u'alias': u'italian', u'title': u'Italian'},
  {u'alias': u'bakeries', u'title': u'Bakeries'},
  {u'alias': u'breakfast_brunch', u'title': u'Breakfast & Brunch'}],
 u'coordinates': {u'latitude': 34.0469300995766,
  u'longitude': -118.256601457672},
 u'display_phone': u'(213) 802-1470',
 u'hours': [{u'hours_type': u'REGULAR',
   u'is_open_now': False,
   u'open': [{u'day': 0,
     u'end': u'2200',
     u'is_overnight': False,
     u'start': u'0700'},
    {u'day': 1, u'end': u'2200', u'is_overnight': False, u'start': u'0700'},
    {u'day': 2, u'end': u'2200', u'is_overnight': False, u'start': u'0700'},
    {u'day': 3, u'end': u'2200', u'is_overnight': False, u'start': u'0700'},
    {u'day': 4, u'end': u'2300', u'is_overnight': False, u'start': u'0700'},
    {u'day': 5, u'end': u'2300', u'is_overnight': False, u'start': u'0800'},
    {u'day': 6, u'end': u'2200', u'is_overnight': False, u'start': u'0800'}]}],
 u'id': u'TkFEKhsCixPW